In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import itertools

In [2]:
%cd projet

/home/phillyflingo/Desktop/ECM/M2 IAAA/temps2/SAM/projet


In [3]:
import keras

import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, ActivityRegularization, Lambda, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.layers import AveragePooling2D, Input
from keras import backend as K 
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

import os
import glob
import time

import sklearn
from sklearn.preprocessing import normalize

from keras.applications.vgg19 import VGG19
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.utils import * 

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

_msdi_path = 'msdi'  # Change this to configure your path to MSDI dataset


def get_msdi_dataframe(msdi_path=_msdi_path):
    return pd.read_csv(Path(msdi_path) / 'msdi_mapping.csv')


def load_mfcc(entry, msdi_path=_msdi_path):
    x = np.load(Path(msdi_path) / entry['mfcc'])
    return x[entry['msd_track_id']]


def load_img(entry, msdi_path=_msdi_path):
    return plt.imread(Path(msdi_path) / entry['img'])


def load_deep_audio_features(entry, msdi_path=_msdi_path):
    subset_file = 'X_{}_audio_MSD-I.npy'.format(entry['set'])
    x = np.load(Path(msdi_path) / 'deep_features' / subset_file)
    idx = entry['deep_features']
    return x[idx, :]


def get_set(entry):
    return entry['set']


def get_label(entry):
    return entry['genre']


def get_label_list(msdi_path=_msdi_path):
    df = pd.read_csv(Path(msdi_path) / 'labels.csv', header=None)
    return list(df.iloc[:, 0])

In [5]:
if __name__ == '__main__':
    # Exemple d'utilisation
    msdi = get_msdi_dataframe(_msdi_path)
    print('Dataset with {} entries'.format(len(msdi)))
    print('#' * 80)
    print('Labels:', get_label_list())
    print('#' * 80)

    entry_idx = 100
    one_entry = msdi.loc[entry_idx]
    print('Entry {}:'.format(entry_idx))
    print('#' * 80)
    img = load_deep_audio_features(one_entry, _msdi_path)
    print(img[1300])

Dataset with 30712 entries
################################################################################
Labels: ['Blues', 'Country', 'Electronic', 'Folk', 'Jazz', 'Latin', 'Metal', 'New Age', 'Pop', 'Punk', 'Rap', 'Reggae', 'RnB', 'Rock', 'World']
################################################################################
Entry 100:
################################################################################
0.0


In [8]:
import gc
def get_X_y(msdi_path):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    mfccs_lenghts = []
    cpt = 0
    msdi = get_msdi_dataframe(msdi_path)
    for idx in range(len(msdi)):
        entry = msdi.loc[idx]
        cpt+=1
        if  get_set(entry) =='train' :
            x = load_deep_audio_features(entry, msdi_path)
            X_train.append(x)
            x=None
            y = get_label(entry)
            y_train.append(y)
            if cpt/10 == 0:
                print(cpt)
        if  get_set(entry) =='test' :
            print(cpt)
            x = load_deep_audio_features(entry, msdi_path)
            X_test.append(x)
            x=None
            y = get_label(entry)
            y_test.append(y)
        gc.collect()
    print('end')
    X_train = np.array(X_train,dtype=object)
    y_train = np.array(y_train)
    X_test = np.array(X_test,dtype=object)
    y_test = np.array(y_test)
    return X_train,y_train,X_test,y_test

_msdi_path = 'msdi'  # Change this to configure your path to MSDI dataset
            
X_train,y_train,X_test,y_test = get_X_y(_msdi_path)


KeyboardInterrupt: 

In [ ]:
np.save('X_train_deep.npy',X_train)
np.save('y_train_deep.npy',y_train)
np.save('X_test_deep.npy',X_test)
np.save('y_test_deep.npy',y_test)

In [ ]:
X_train.shape

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
#X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_train.shape


In [ ]:
#y_train, y_test = prepare_targets(y_train, y_test)

In [ ]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

labelencoder = LabelEncoder()
y_train = y_train.reshape(-1, 1)
#y_test = y_test.reshape(-1, 1)

labelencoder.fit(y_train)
#labelencoder.fit(y_test)


y_train = labelencoder.transform(y_train)
#y_test = labelencoder.transform(y_test)

In [ ]:
type(y_train)

In [ ]:

input_shape = (X_train.shape[1], 1)
input_shape
# Layer 1
model = Sequential()
model.add(Conv1D(64, 1, activation='relu', input_shape=input_shape))
model.add(MaxPooling1D(2))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(15, activation='softmax'))
# compile the model
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=8, verbose=2)

In [ ]:
def generator(features, labels, batch_size):
    while True:
        batch_features = []
        batch_labels = []
        
        for i in range(batch_size):
            # choose random index in features
            index = np.random.choice(len(features),1)
            batch_features.extend(features[index])
            batch_labels.extend(labels[index])
        batch_features = np.array(batch_features)
        batch_labels = np.array(batch_labels)
        yield batch_features, batch_labels    

In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels):
    """
    plots the confusion matrix
    """
    matrix = confusion_matrix(y_true, y_pred)    
    fig, ax = plt.subplots(figsize=(12,10))
    plt.imshow(matrix)
    ax.set_xticks(range(len(labels)));
    ax.set_xticklabels(labels, rotation=0)
    ax.set_yticks(range(len(labels)));
    ax.set_yticklabels(labels)
    max_confusions = 0
    confused_classes = (-1, -1)
    for i, true_label in enumerate(matrix):
        for j, predicted_label in enumerate(true_label):
            text = ax.text(j, i, matrix[i, j],
                        ha="center", va="center", color="w");
    plt.tick_params(axis=u'both', which=u'both',length=0)
    plt.title("Confusion Matrix")

In [ ]:
y_train

In [ ]:
def plot_histogram(array, bins=1000, title="Histogram", xlabel="x", ylabel="y"):
    array  = np.array(array)
    values, counts = np.unique(array, return_counts=True)

    
    plt.hist(array)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()
plot_histogram(mfccs_lenghts)

## Genre Classification : MFCC Features with CNN  

In [ ]:
"""
x = Input(shape=input_shape, name='input')
y0 = AveragePooling2D((14,14), padding='same')(x)# MaxPooling2D   AveragePooling2D ##### You may also use GlobalAveragePooling layers

y0 = Lambda(lambda x: K.l2_normalize(x, axis=-1))(x)
y0 = Flatten()(y0)

y1 = Dense(n_classes, activation='softmax')(y0)

model1 = Model(inputs=x,outputs=y1)

#Combine the networks
model1.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

print(model1.summary())"""

In [ ]:
"""X_train = np.asarray(X_train).astype(np.float32)
#y_train = np.asarray(y_train).astype(np.float32)
X_test = np.asarray(X_train).astype(np.float32)
#y_test = np.asarray(y_train).astype(np.float32)"""

In [ ]:
"""# reshape data to fit model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

# one hot encode target values
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
"""